In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\ilakk\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\ilakk\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [3]:
pip install matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\ilakk\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
customers = pd.read_csv("C:/Users/ilakk/Downloads/Customers.csv")

In [7]:
products = pd.read_csv("C:/Users/ilakk/Downloads/Products.csv")

In [8]:
transactions = pd.read_csv("C:/Users/ilakk/Downloads/Transactions.csv")

In [9]:
# Feature Engineering
# Aggregate transaction features (e.g., Total Spent, Number of Transactions, etc.)
transaction_features = transactions.groupby('CustomerID').agg(
    TotalSpent=('TotalValue', 'sum'),
    NumTransactions=('TransactionID', 'count')
).reset_index()


In [10]:
# Merge customer profile information and transaction features
data = pd.merge(customers, transaction_features, on='CustomerID', how='inner')


In [12]:
# Check column names in the 'Customers' dataframe
print(customers.columns)

# Preview the data to see the structure
print(customers.head())


Index(['CustomerID', 'CustomerName', 'Region', 'SignupDate'], dtype='object')
  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  10-07-2022
1      C0002      Elizabeth Lutz           Asia  13-02-2022
2      C0003      Michael Rivera  South America  07-03-2024
3      C0004  Kathleen Rodriguez  South America  09-10-2022
4      C0005         Laura Weber           Asia  15-08-2022


In [17]:
print(transactions.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price'],
      dtype='object')


In [18]:
# Feature Engineering: Calculate TotalSpent and NumTransactions for each customer
customer_features = transactions.groupby('CustomerID').agg(
    TotalSpent=('TotalValue', 'sum'),  # Sum of 'TotalValue' for each customer
    NumTransactions=('TransactionID', 'count')  # Count of 'TransactionID' for each customer
).reset_index()

# Check the first few rows of the features DataFrame
print(customer_features.head())


  CustomerID  TotalSpent  NumTransactions
0      C0001     3354.52                5
1      C0002     1862.74                4
2      C0003     2725.38                4
3      C0004     5354.88                8
4      C0005     2034.24                3


In [19]:
# Merge the calculated features with the customer information
customer_data = pd.merge(customers[['CustomerID', 'Region']], customer_features, on='CustomerID')


In [20]:
# One-hot encode the 'Region' column to include it in the clustering
customer_data = pd.get_dummies(customer_data, columns=['Region'], drop_first=True)

In [22]:
# Normalize the features using StandardScaler
scaler = StandardScaler()
# Drop 'CustomerID' column as it is not needed for clustering
features = customer_features.drop(columns=['CustomerID'])
# Scale the features
scaled_features = scaler.fit_transform(features)

# Check the scaled features
print(scaled_features[:5])  # Preview the scaled features


[[-0.06170143 -0.01145819]
 [-0.87774353 -0.46749414]
 [-0.40585722 -0.46749414]
 [ 1.03254704  1.35664965]
 [-0.78392861 -0.92353008]]


In [23]:
# Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42)
customer_data['Cluster'] = kmeans.fit_predict(scaled_features)

In [24]:
# Calculate Davies-Bouldin Index for clustering evaluation
db_index = davies_bouldin_score(scaled_features, customer_data['Cluster'])
print(f"DB Index: {db_index}")


DB Index: 0.8595340221510472
